In [63]:
!pip install -U deep-translator
!pip install vaderSentiment

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
# from googletrans import Translator
import re
import nltk
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from deep_translator import GoogleTranslator
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

In [12]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [133]:
# nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/zaens/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Import Data

In [127]:
df = pd.read_csv("youtube_comments.csv", index_col="Unnamed: 0")
df.head()

,author,published_at,comment,like
0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,Ikn di buka untuk umum ...indah nya kota ikn d...,0
1,@daryanto07,2024-08-01T00:45:05Z,"Masuk akal nggak sih, mulai dibangun tahun 202...",0
2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,Ikn jalan terus hebat .semangat maju terus ind...,0
3,@harunindrapratama5497,2024-07-18T18:14:51Z,Orang bodoh penjilat,0
4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0


# Translate

In [140]:
def translate_comment(comment):
    if len(comment) < 5000:
        return GoogleTranslator(source='auto', target='en').translate(comment)
    else:
        sentences = sent_tokenize(comment)
        return " ".join([GoogleTranslator(source='auto', target='en').translate(sentence) for sentence in sentences])

In [141]:
df["en_comment"] = df.comment.apply(translate_comment)

In [142]:
# df.to_csv("en_comments.csv")

In [143]:
df = pd.read_csv("en_comments.csv")
df.head()

,Unnamed: 0,author,published_at,comment,like,en_comment
0,0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,Ikn di buka untuk umum ...indah nya kota ikn d...,0,Ikn is open to the public...the beauty of the ...
1,1,@daryanto07,2024-08-01T00:45:05Z,"Masuk akal nggak sih, mulai dibangun tahun 202...",0,"Does it make sense, starting construction in 2..."
2,2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,Ikn jalan terus hebat .semangat maju terus ind...,0,Ikn keep going great. Keep up the spirit of pr...
3,3,@harunindrapratama5497,2024-07-18T18:14:51Z,Orang bodoh penjilat,0,A sycophantic fool
4,4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree


# Preprocessing

## Drop Missing Value

In [144]:
df.isnull().sum()

Unnamed: 0       0
author           1
published_at     0
comment          0
like             0
en_comment      20
dtype: int64

In [194]:
df.dropna(inplace=True)

In [206]:
df.isnull().sum()

author          0
published_at    0
comment         0
like            0
en_comment      0
dtype: int64

In [147]:
df.head()

,Unnamed: 0,author,published_at,comment,like,en_comment
0,0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,Ikn di buka untuk umum ...indah nya kota ikn d...,0,Ikn is open to the public...the beauty of the ...
1,1,@daryanto07,2024-08-01T00:45:05Z,"Masuk akal nggak sih, mulai dibangun tahun 202...",0,"Does it make sense, starting construction in 2..."
2,2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,Ikn jalan terus hebat .semangat maju terus ind...,0,Ikn keep going great. Keep up the spirit of pr...
3,3,@harunindrapratama5497,2024-07-18T18:14:51Z,Orang bodoh penjilat,0,A sycophantic fool
4,4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree


## Cleaning

In [148]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphanumeric characters
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

In [149]:
df.en_comment = df.en_comment.apply(clean_text)
df.comment = df.comment.apply(clean_text)

In [150]:
df.head()

,Unnamed: 0,author,published_at,comment,like,en_comment
0,0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,ikn di buka untuk umum indah nya kota ikn di t...,0,ikn is open to the publicthe beauty of the ikn...
1,1,@daryanto07,2024-08-01T00:45:05Z,masuk akal nggak sih mulai dibangun tahun punc...,0,does it make sense starting construction in pe...
2,2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,ikn jalan terus hebat semangat maju terus indo...,0,ikn keep going great keep up the spirit of pro...
3,3,@harunindrapratama5497,2024-07-18T18:14:51Z,orang bodoh penjilat,0,a sycophantic fool
4,4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree


In [202]:
# df.to_csv("clean_comments.csv", index=False)

In [203]:
df = pd.read_csv("clean_comments.csv")
df.head()

,author,published_at,comment,like,en_comment
0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,ikn di buka untuk umum indah nya kota ikn di t...,0,ikn is open to the publicthe beauty of the ikn...
1,@daryanto07,2024-08-01T00:45:05Z,masuk akal nggak sih mulai dibangun tahun punc...,0,does it make sense starting construction in pe...
2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,ikn jalan terus hebat semangat maju terus indo...,0,ikn keep going great keep up the spirit of pro...
3,@harunindrapratama5497,2024-07-18T18:14:51Z,orang bodoh penjilat,0,a sycophantic fool
4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree


## Labelling 

In [204]:
def labeling(comment):
    sentiment = SentimentIntensityAnalyzer()
    x = sentiment.polarity_scores(comment)
    # print(x)
    if x["compound"] >= 0:
        return "positif"
    else:
        return "negatif"
    # return "positif" if x["compound"] >= 0 else "negatif"

In [207]:
df["label"] = df["en_comment"].apply(labeling)

In [208]:
df.head()

,author,published_at,comment,like,en_comment,label
0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,ikn di buka untuk umum indah nya kota ikn di t...,0,ikn is open to the publicthe beauty of the ikn...,positif
1,@daryanto07,2024-08-01T00:45:05Z,masuk akal nggak sih mulai dibangun tahun punc...,0,does it make sense starting construction in pe...,positif
2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,ikn jalan terus hebat semangat maju terus indo...,0,ikn keep going great keep up the spirit of pro...,positif
3,@harunindrapratama5497,2024-07-18T18:14:51Z,orang bodoh penjilat,0,a sycophantic fool,negatif
4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree,positif


In [209]:
df.label.value_counts()

label
positif    2280
negatif     792
Name: count, dtype: int64

In [210]:
# df.to_csv("label_comment.csv")

In [2]:
df = pd.read_csv("label_comment.csv", index_col="Unnamed: 0")
df.head()

,author,published_at,comment,like,en_comment,label
0,@MaulanaMaulanamaulana-eo5fl,2024-09-20T04:15:25Z,ikn di buka untuk umum indah nya kota ikn di t...,0,ikn is open to the publicthe beauty of the ikn...,positif
1,@daryanto07,2024-08-01T00:45:05Z,masuk akal nggak sih mulai dibangun tahun punc...,0,does it make sense starting construction in pe...,positif
2,@ArwanAhmad-z8g,2024-07-26T02:16:40Z,ikn jalan terus hebat semangat maju terus indo...,0,ikn keep going great keep up the spirit of pro...,positif
3,@harunindrapratama5497,2024-07-18T18:14:51Z,orang bodoh penjilat,0,a sycophantic fool,negatif
4,@istachondoni198,2024-07-18T10:27:29Z,setuju,0,agree,positif


In [3]:
df = df.drop(columns=["author", "published_at", "like", "en_comment"])

In [4]:
df.head()

,comment,label
0,ikn di buka untuk umum indah nya kota ikn di t...,positif
1,masuk akal nggak sih mulai dibangun tahun punc...,positif
2,ikn jalan terus hebat semangat maju terus indo...,positif
3,orang bodoh penjilat,negatif
4,setuju,positif


In [5]:
df.label = df.label.apply(lambda x: 1 if x == "positif" else 0)

# Dataset Splitting

In [7]:
X = df['comment']
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2457,), (615,), (2457,), (615,))

## Tokenizer

In [8]:
tfidf = TfidfVectorizer(tokenizer=word_tokenize, stop_words=sw_indo, token_pattern=None)
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

/home/zaens/miniconda3/envs/MachineLearning/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


In [182]:
print(X_train.shape)

(2457, 8297)


# Dataset & Dataloader

In [9]:
# from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

In [10]:
X_train = X_train.toarray()
X_test = X_test.toarray()

In [13]:
X_train = torch.FloatTensor(X_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_train = torch.FloatTensor(y_train).to(device)
y_test = torch.FloatTensor(y_test).to(device)

In [15]:
bs = 128
train_set = TensorDataset(X_train, y_train)
test_set = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_set, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_set, batch_size=bs)

# Arsitekstur & Config

In [16]:
from jcopdl.layers import linear_block

In [20]:
class GRU(nn.Module):
    def __init__(self, input_size, embedding_dim, output_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_size, embedding_dim, padding_idx=0)
        self.rnn = nn.GRU(embedding_dim, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = linear_block(hidden_size, output_size, activation="sigmoid")
    def forward(self, x, hidden):
        x = self.embedding(x)
        x, hidden = self.rnn(x, hidden)
        x = self.fc(x)
        return x, hidden


In [22]:
config = set_config({
    "input_size" : X_train.shape[1],
    "output_size" : 1,
    "hidden_size" : 64,
    "num_layers" : 2,
    "dropout" : 0,
    "seq_len" : 1,
    "embedding_dim":  64
})

# Training Preparation

In [23]:
model = GRU(config.input_size, config.output_size, config.embedding_dim,config.hidden_size, config.num_layers, config.dropout).to(device)
criterion = nn.BCELoss(reduction="mean")
optimizer = optim.AdamW(model.parameters(), lr=0.001)
callback = Callback(model, config=config, outdir="model/gru")

# Training

In [24]:
from tqdm.auto import tqdm
def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    elif mode == "test":
        model.eval()
    cost = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output, _ = model(feature, None) # feedforward
        output = output.squeeze()
        loss = criterion(output, target)
        if mode == "train":
            loss.backward() # backpropragation
            optimizer.step() # update weight
            optimizer.zero_grad()
        cost += loss.item() * feature.shape[0]
    cost = cost / len(dataset)
    return cost

In [25]:
while True:
    train_cost = loop_fn("train", train_set, train_loader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost = loop_fn("test", test_set, test_loader, model, criterion, optimizer, device)
    # Logging
    callback.log(train_cost, test_cost)
    # Save CheckPoint
    callback.save_checkpoint()

    # Runtime Plotting
    callback.cost_runtime_plotting()
    # Early Stopping
    if callback.early_stopping(model, monitor="test_cost"):
        callback.plot_cost()
        break

Train:   0%|          | 0/20 [00:00<?, ?it/s]

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [212]:
for data in train_loader:
    print(data)
    break

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        1, 0, 1, 1, 1, 1, 0, 0], device='cuda:0')]
